--- Day 13: Transparent Origami ---

You reach another volcanically active part of the cave. It would be nice if you could do some kind of thermal imaging so you could tell ahead of time which caves are too hot to safely enter.

Fortunately, the submarine seems to be equipped with a thermal camera! When you activate it, you are greeted with:

Congratulations on your purchase! To activate this infrared thermal imaging
camera system, please enter the code found on page 1 of the manual.

Apparently, the Elves have never used this feature. To your surprise, you manage to find the manual; as you go to open it, page 1 falls out. It's a large sheet of transparent paper! The transparent paper is marked with random dots and includes instructions on how to fold it up (your puzzle input). For example:

    6,10
    0,14
    9,10
    0,3
    10,4
    4,11
    6,0
    6,12
    4,1
    0,13
    10,12
    3,4
    3,0
    8,4
    1,10
    2,14
    8,10
    9,0

    fold along y=7
    fold along x=5

In [12]:
'''
General instructions:
- Create a grid x by y with a starting point (0,0) and an ending point (x,y)
- The grid is a 2D array of dots and #s
- We need to find a way fold the grid into halves until we finish the folds

Functions:
- read_grid: Reads the input for the grid from the text file
- create_grid: creates a grid of dots and #s
- fold_grid: folds the grid into halves until we finish the folds
'''

'\nGeneral instructions:\n- Create a grid x by y with a starting point (0,0) and an ending point (x,y)\n- The grid is a 2D array of dots and #s\n- We need to find a way fold the grid into halves until we finish the folds\n\nFunctions:\n- read_grid: Reads the input for the grid from the text file\n- create_grid: creates a grid of dots and #s\n- fold_grid: folds the grid into halves until we finish the folds\n'

In [13]:
import util
import os
import numpy as np
import re

os.chdir('/Users/andrescrucettanieto/Documents/GitHub/advent_of_code/2021')

In [14]:
def read_input(filename):
    '''
    Reads the input for the grid from the text file
    '''
    input_lst = util.read_strs(filename)
    directions_start = input_lst.index("fold")
    
    # Split the input and the directions
    input = input_lst[:directions_start]
    directions = input_lst[directions_start:]
    
    return input, directions

In [15]:
def clean_input(input):
    '''
    Converts input into pair of two integers
    '''
    clean_input = []
    for pair in input:
        x,y = pair.split(',')
        clean_input.append((int(x), int(y)))
    return clean_input

In [16]:
def clean_directions(directions):
    '''
    Converts directions into a list of tuples
    '''
    clean_directions = []
    directions = [e for e in directions if e not in ('fold', "along")]
    for direction in directions:
        axis,units = direction.split("=")
        clean_directions.append((axis, int(units)))
    return clean_directions

In [17]:
def get_boundaries(input):
    '''
    Gets the max x and y from the input
    '''
    max_x = max([x for (x, _) in input])
    max_y = max([y for (_, y) in input])
    return max_x, max_y

In [18]:
def create_grid(input):
    '''
    Creates a grid of dots and #s
    '''
    max_x, max_y = get_boundaries(input)
    grid = np.zeros((max_y+1, max_x+1), dtype=np.int8)
    for x,y in input:
        grid[y][x] = 1
    return grid

In [26]:
def fold_paper(grid, directions, fold_count, fold_all=False):
    '''
    Folds the grid into halves until we finish the folds
    '''
    if fold_all:
        fold_count = len(directions)
    for i in range(fold_count):
        axis, units = directions[i]
        if axis == 'y':
            up = grid[:units]
            down = grid[units+1:]
            arr = up + np.flipud(down)
        
        elif axis == 'x':
            left = grid[:,:units]
            right = grid[:,units+1:]
            # Dealing with uneven folds
            if left.shape != right.shape:
                diff = abs(left.shape[1] - right.shape[1])
                buffer = np.zeros((right.shape[0], diff), dtype=np.int8)
                right = np.hstack((right, buffer))
                
            arr = left + np.fliplr(right)
        grid = arr
    return arr

In [27]:
def task1(input,directions, fold_count=0, fold_all=False):
    '''
    Finding the number of 1s and above 1s in the grid.
    '''
    # Cleaning the input and directions
    clean_inp = clean_input(input)
    clean_dir = clean_directions(directions)
    
    # Creating the grid
    grid = create_grid(clean_inp)
    
    # Folding the grid
    grid = fold_paper(grid, clean_dir,fold_count, fold_all)
    
    # Counting the number of 1
    ones = np.count_nonzero(grid == 1)
    above_ones = np.count_nonzero(grid > 1)
    total = ones + above_ones
    return grid,total

In [44]:
def task2(input,directions):
    '''
    Finding the shape of all the folds.
    '''
    grid, _ = task1(input,directions,0, fold_all=True)
    folded_grid = np.where(grid > 1, "█", "_")
    return folded_grid
    

In [45]:
input, directions = read_input('data/13_test.txt')

In [46]:
input_full, directions_full = read_input('data/13.txt')

In [47]:
task1(input, directions, 1,fold_all=False)

(array([[1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0],
        [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 1, 0, 0, 1, 0, 2, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8),
 17)

In [48]:
grid,ones = task1(input_full, directions_full, 1,fold_all=True)

In [49]:
task2_arr = task2(input_full, directions_full)

In [53]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [60]:
print(task2_arr.tolist())

[['_', '_', '█', '█', '_', '█', '_', '█', '_', '_', '█', '█', '█', '█', '_', '_', '█', '█', '_', '_', '█', '_', '_', '_', '_', '_', '█', '█', '_', '_', '█', '_', '_', '█', '_', '█', '_', '█', '_', '_'], ['_', '_', '_', '█', '_', '█', '_', '_', '█', '_', '_', '_', '_', '█', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_'], ['_', '_', '_', '█', '_', '█', '_', '_', '█', '_', '_', '_', '█', '_', '_', '█', '_', '_', '_', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_', '█', '_', '_', '█', '_'], ['_', '_', '_', '█', '_', '█', '█', '█', '_', '_', '_', '█', '_', '_', '_', '█', '_', '_', '_', '_', '█', '_', '_', '█', '_', '█', '█', '█', '█', '_', '█', '_', '_', '█', '_', '█', '█', '█', '_', '_'], ['█', '_', '_', '█', '_', '█', '_', '_', '_', '_', '_', '_', '_', '_', '_', '█', '_', '_', '█', '_', '█', '_', '_', '_', '_', '█', '_', '_', '█', '_', '█', '_', '_', '_', '_', '█', '_', '█', 